# 01_simulations

This notebook demonstrates the **Simulation** capabilities of the `pysubmit` package. Specifically, it walks through:

1. Running an **Eigenmode** simulation
2. Inspecting and saving simulation results
3. Serialize and save results
4. Adjusting simulation parameters
5. Serializing and deserializing simulation objects

> **Note**:
> The `pysubmit.simulation` module supports multiple simulation classes.
> In this tutorial, we focus on one: `EigenmodeAnalysis`.

## Prerequisites
- Make sure `pyaedt` and `pysubmit` are installed and accessible.
- We assume an existing HFSS project file is available at `./designs/simple_design.aedt`.

### Step 1: Imports and Setup

In [1]:
from pysubmit.simulation import EigenmodeAnalysis
from ansys.aedt.core.hfss import Hfss

# HFSS project path
hfss_file_path = './designs/simple_design.aedt'

### Step 2: Open Project and Run an Eigenmode Simulation
We�ll open an existing HFSS project using `pyaedt`, then use it to create and run an eigenmode simulation.

In [2]:
# Create and run an eigenmode simulation
with Hfss(
    project=hfss_file_path,
    design='my_design',
    remove_lock=True,
    close_on_exit=True
) as hfss:
    simulation = EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')
    result = simulation.analyze(hfss)
    # The PyAEDT logs may appear here; they are normal and can be ignored unless debugging.

PyAEDT INFO: Parsing ./designs/simple_design.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.13.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: File ./designs/simple_design.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_e51ae643-13fa-4bfd-a9f4-1b116029b696.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 54319
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 13048.
PyAEDT INFO: Project simple_design has been opened.
PyAEDT INFO: Aedt Objects correctly read
PyAEDT INFO: Python version 3

### Step 3: Inspect the Simulation Results
Once the simulation is complete, it returns a **result** object with:
- Printable summary
- JSON-serializable structure

Let�s print the first mode�s frequency and Q-factor.

In [3]:
# Access mode 1 results
print("Eigenmode Simulation Results:")
print(result)
print(f"Mode 1 frequency: {result.results[1].frequency}")
print(f"Mode 1 Q-factor: {result.results[1].quality_factor}")

Eigenmode Simulation Results:
type=<SimulationOutputTypesNames.EIGENMODE_RESULT: 'eigenmode_result'> results={1: SingleModeResult(mode_number=1, quality_factor=0.0, frequency=Value(value=7.939595892830551, unit='GHz'), label=None)} frequencies_unit='GHz'
Mode 1 frequency: value=7.939595892830551 unit='GHz'
Mode 1 Q-factor: 0.0


### Step 4: Serialize and Save Results
You can serialize the results into JSON format for logging or further processing.

In [4]:
result_as_json = result.model_dump_json(indent=4)
print("Serialized Results:\n", result_as_json)

# Save the JSON to a file
with open('my_results.json', 'w') as f:
    f.write(result_as_json)

Serialized Results:
 {
    "type": "eigenmode_result",
    "results": {
        "1": {
            "mode_number": 1,
            "quality_factor": 0.0,
            "frequency": {
                "value": 7.939595892830551,
                "unit": "GHz"
            },
            "label": null
        }
    },
    "frequencies_unit": "GHz"
}


### Step 5: Adjust Simulation Setup Parameters
You can modify parameters before running the simulation if you need different passes, frequency ranges, etc.

In [5]:
# Example simulation parameters
simulation.setup_parameters = {
    'MaximumPasses': 4,
    'MinimumPasses': 1,
    'NumModes': 3,
    'MinimumFrequency': '2GHz',
    # More parameters...
}
print("Updated simulation parameters:", simulation.setup_parameters)

Updated simulation parameters: {'MaximumPasses': 4, 'MinimumPasses': 1, 'NumModes': 3, 'MinimumFrequency': '2GHz'}


### Step 6: Serialize and Reload the Simulation Object
In some workflows, you may want to reuse the same simulation configuration across different designs or sessions.

In [6]:
simulation_as_string = simulation.model_dump_json(indent=4)
print("Serialized Simulation Object:\n", simulation_as_string)

# Save to disk
with open('my_simulation.json', 'w') as f:
    f.write(simulation_as_string)

# Reload
with open('my_simulation.json', 'r') as f:
    data = f.read()

simulation_loaded = EigenmodeAnalysis.model_validate_json(data)
print("Loaded Simulation:", simulation_loaded)

Serialized Simulation Object:
 {
    "type": "eigenmode",
    "setup_name": "Setup1",
    "design_name": "my_design",
    "cores": 4,
    "gpus": 0,
    "setup_parameters": {
        "MaximumPasses": 4,
        "MinimumPasses": 1,
        "NumModes": 3,
        "MinimumFrequency": "2GHz"
    }
}
Loaded Simulation: type=<SimulationTypesNames.EIGENMODE: 'eigenmode'> setup_name='Setup1' design_name='my_design' cores=4 gpus=0 setup_parameters={'MaximumPasses': 4, 'MinimumPasses': 1, 'NumModes': 3, 'MinimumFrequency': '2GHz'}


## Conclusion

In this tutorial, we:

- Set up and ran an **eigenmode simulation** using `pysubmit.simulation.EigenmodeAnalysis`.
- Explored the **results object**, showing frequencies and Q-factors.
- Demonstrated how to **serialize** results and simulation objects for reuse.

In the following tutorials, you'll see how these simulation objects combine with other features like sweepers, builders, and data handlers to create powerful automated workflows.